In [41]:
import os
import re
import pandas as pd
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Model, save_model, Sequential
from tensorflow.keras.layers import BatchNormalization, Input, Conv2D, Conv3D, Flatten, GlobalAveragePooling2D, GlobalAveragePooling3D, AveragePooling2D, AveragePooling3D
from tensorflow.keras.optimizers import SGD, Adamax, Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [42]:
images = []
classes = []

In [43]:
# read images from 'with_mask'/'without_mask' folder
for root, _, files in os.walk('.\with_mask'):
    for file in files:
        if re.search(r'\.(jpeg|jpg|png)', file):
            imagePath = os.path.join(root, file)
            image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (256, 256))
            image = image/256
            images.append(image)
            if 'without_mask' in imagePath:
                classes.append(0)
            elif 'with_mask' in imagePath:
                classes.append(1)
                
images = np.array(images)
classes = np.array(classes)

In [44]:
# transformedImagesPath = '.\Transformed_With_Mask'

In [45]:
# imgGenerator = ImageDataGenerator(featurewise_center=False,
#                                   samplewise_center=False,
#                                   featurewise_std_normalization=False,
#                                   samplewise_std_normalization=False,
#                                   zca_whitening=False,
#                                   zca_epsilon=2e-03,
#                                   rotation_range=40,
#                                   width_shift_range=10.0,
#                                   height_shift_range=10.0,
#                                   brightness_range=None,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   channel_shift_range=0.1,
#                                   fill_mode="nearest",
#                                   cval=0.0,
#                                   horizontal_flip=True,
#                                   vertical_flip=True,
#                                   preprocessing_function=None,
#                                   data_format=None,
#                                   validation_split=0.0,
#                                   dtype=np.array,
#                                  rescale=1.0/256)

In [46]:
# a = imgGenerator.flow_from_directory(transformedImagesPath, 
#                                      batch_size=10, 
#                                      target_size=(256, 256))

In [47]:
# b,c = a.next()

In [48]:
# imgGenerator.flow(b, c, batch_size=20)

In [49]:
# X_train, X_test, y_train, y_test = train_test_split(X_trans, y_trans, train_size=0.7, random_state=42)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(images, classes, train_size=0.7, random_state=42)

In [51]:
model = Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

start = time.time()
#model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2,callbacks=[checkpoint])
model.fit(images, classes, batch_size=32, epochs=5, validation_split=0.35, callbacks=[checkpoint], shuffle=True)

Train on 448 samples, validate on 242 samples
Epoch 1/5
448/448 [==============================] - 254s 568ms/sample - loss: 0.0441 - accuracy: 0.9844 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
448/448 [==============================] - 247s 552ms/sample - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
448/448 [==============================] - 219s 488ms/sample - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
448/448 [==============================] - 216s 482ms/sample - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
448/448 [==============================] - 229s 510ms/sample - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [53]:
end = time.time()
print('Time taken to fit the model',(end-start))

Time taken to fit the model 1165.8081638813019


In [54]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
size = 4
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
webcam = cv2.VideoCapture(0)
while True:
    (rval, image) = webcam.read()
    image = cv2.flip(image, 1, 1) #Flip to act as a mirror
    im = image
    im = im / 256
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    mini = cv2.resize(im, (256, 256))
    faces = classifier.detectMultiScale(mini)
    
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        #resized=cv2.resize(face_img,(256,256))
        #normalized=resized/255.0
        reshaped=np.reshape(face_img,(1,256,256,3))
        result=model.predict(normalized)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        

    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

In [15]:
webcam = cv2.VideoCapture(0)
(rval, im) = webcam.read()

In [36]:
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

In [25]:
webcam = cv2.VideoCapture(0)
while True:
    (rval, image) = webcam.read()
    im = image
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im=cv2.flip(im,1,1) #Flip to act as a mirror
    #im = im / 256
    mini = cv2.resize(im, (256, 256))
    faces = classifier.detectMultiScale(mini)
    
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        #resized=cv2.resize(face_img,(256,256))
        #normalized=resized/255.0
        reshaped=np.reshape(face_img,(1,256,256,3))
        result=model.predict(normalized)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        

    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 0 into shape (1,256,256,3)

In [ ]:
# webcam = cv2.VideoCapture(0)
# while True:
#     (rval, im) = webcam.read()
#     im=cv2.flip(im,1,1) #Flip to act as a mirror
# 
#     # Show the image
#     cv2.imshow('LIVE',   im)
#     key = cv2.waitKey(10)
#     # if Esc key is press then break out of the loop 
#     if key == 27: #The Esc key
#         break
# # Stop video
# webcam.release()
# 
# # Close all started windows
# cv2.destroyAllWindows()